In [1]:
import pandas as pd

In [2]:
# FIX SSL: CERTIFICATE_VERIFY_FAILED with Python3
# https://stackoverflow.com/a/49174340
import ssl

ssl._create_default_https_context = ssl._create_unverified_context

In [3]:
url = 'https://www.juntadeandalucia.es/institutodeestadisticaycartografia/badea/stpivot/stpivot/Print?cube=013d884c-cfc5-4c21-9e78-e26225682afd&type=3&foto=si&ejecutaDesde=&codConsulta=38668&consTipoVisua=JP'
fields = ['Lugar de residencia', 'Medida', 'Valor']
df = pd.read_csv(url, delimiter=';', usecols=fields, decimal=',')

In [4]:
df

,Lugar de residencia,Medida,Valor
0,Huelva,Población,510743
1,Huelva,Confirmados PDIA,2095
2,Huelva,Confirmados PDIA 14 días,578
3,Huelva,Tasa PDIA 14 días,"113,16846241651868"
4,Huelva,Confirmados PDIA 7 días,292
...,...,...,...
627,Municipio de Huelva sin especificar,Tasa PDIA 14 días,-
628,Municipio de Huelva sin especificar,Confirmados PDIA 7 días,5
629,Municipio de Huelva sin especificar,Total Confirmados,34
630,Municipio de Huelva sin especificar,Curados,4


In [5]:
df['Valor'] = pd.to_numeric(df['Valor'],errors='coerce')

In [6]:
dfAnterior = pd.read_csv('covid_huelva.csv', index_col='Lugar de residencia')

In [7]:
# Confirmados PCR provincia Huelva
p = df[df['Lugar de residencia'] == 'Huelva'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en la provincia de Huelva: ', p)

Confirmados PCR en la provincia de Huelva:  2095.0


In [8]:
# Confirmados PCR Huelva capitalpd
c = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en Huelva capital: ', c)

Confirmados PCR en Huelva capital:  562.0


In [9]:
c7 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 7 días']['Valor'].values[0]
c14 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 14 días']['Valor'].values[0]
p = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Población']['Valor'].values[0]

In [10]:
print('Huelva (capital)\n')
#print('Confirmados PCR 7 días x 1,000 habitantes: ', f"{c7/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c7:.0f}", 'personas en los últimos 7 días', '\n')
#print('Confirmados PCR 14 días x 1,000 habitantes: ', f"{c14/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c14:.0f}", 'personas en los últimos 14 días')

Huelva (capital)

Un positivo PCR cada 1260 personas en los últimos 7 días 

Un positivo PCR cada 726 personas en los últimos 14 días


In [11]:
filtro = ((df['Medida'] == 'Confirmados PDIA')| (df['Medida'] == 'Confirmados PDIA 7 días') | (df['Medida'] == 'Población' ) | (df['Medida'] == 'Confirmados PDIA 14 días')) & (df['Valor'] > 0)
tabla = pd.pivot_table(df.loc[filtro], values='Valor', index='Lugar de residencia', columns='Medida').dropna()

In [12]:
# Reorder columns
tabla = tabla[['Confirmados PDIA', 'Confirmados PDIA 7 días', 'Confirmados PDIA 14 días', 'Población']]

In [13]:
tabla['tasa7'] = tabla['Confirmados PDIA 7 días']/tabla['Población']*100000
tabla['tasa14'] = tabla['Confirmados PDIA 14 días']/tabla['Población']*100000

In [14]:
tabla['Diferencia PDIA'] = tabla['Confirmados PDIA'] - dfAnterior['Confirmados PDIA']

In [15]:
# Nuevos casos desde la última captura
tabla[tabla['Diferencia PDIA'] != 0].sort_values('Diferencia PDIA', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Huelva,2095.0,292.0,578.0,510743.0,57.171611,113.168462,46.0
Huelva-Costa,1085.0,174.0,318.0,288115.0,60.392552,110.372594,27.0
Huelva (capital),562.0,114.0,198.0,143663.0,79.352373,137.822543,14.0
Lepe,157.0,24.0,49.0,27431.0,87.492253,178.630017,10.0
Condado-Campiña,705.0,79.0,190.0,155057.0,50.949006,122.535584,9.0
Sierra de Huelva-Andévalo Central,271.0,34.0,59.0,67571.0,50.317444,87.315564,7.0
Cortegana,22.0,10.0,12.0,4666.0,214.316331,257.179597,6.0
Gibraleón,43.0,9.0,17.0,12607.0,71.388911,134.845721,4.0
Almonte,69.0,9.0,15.0,24191.0,37.203919,62.006531,2.0


In [16]:
if tabla[tabla['Diferencia PDIA'] != 0]['Diferencia PDIA'].sum() > 0:
  # Save to csv
  tabla.to_csv('covid_huelva.csv', encoding='utf-8', header=True, index=True)  

In [17]:
tabla.sort_values('tasa14', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Granado (El),13.0,11.0,11.0,516.0,2131.782946,2131.782946,0.0
Villarrasa,27.0,11.0,19.0,2176.0,505.514706,873.161765,0.0
San Juan del Puerto,74.0,7.0,38.0,9300.0,75.268817,408.602151,0.0
Jabugo,14.0,1.0,9.0,2250.0,44.444444,400.000000,0.0
Palma del Condado (La),104.0,18.0,34.0,10761.0,167.270700,315.955766,2.0
Cortegana,22.0,10.0,12.0,4666.0,214.316331,257.179597,6.0
Zalamea la Real,28.0,3.0,7.0,3068.0,97.783572,228.161669,0.0
Manzanilla,20.0,1.0,4.0,2135.0,46.838407,187.353630,0.0
Lepe,157.0,24.0,49.0,27431.0,87.492253,178.630017,10.0
